In [1]:
import os
import re
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, ClassLabel

from langchain.document_loaders import PyPDFLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from neo4j import GraphDatabase
from utils import read_json

import torch

In [2]:
def get_whole_pdf(file_data):
    whole_pdf = ""
    for i in range(len(file_data)):
        whole_pdf += file_data[i].page_content

    return whole_pdf

In [3]:
configs = read_json('configs.json')
DATA_PATH = configs["DATA_PATH"]
DB_PATH = configs['DB_PATH']
files_list = [file for file in os.listdir(DATA_PATH) if os.path.isfile(os.path.join(DATA_PATH, file))]

def read_pdf(files_list, data_path=DATA_PATH):
    content_dict = {}

    for file_name in files_list:
        file_path = data_path + file_name
        loader = PyPDFLoader(file_path)
        content = loader.load()
        whole_pdf = ""
        for i in range(len(content)):
            whole_pdf += content[i].page_content

        whole_pdf = re.sub(r'[：。\n]', '', whole_pdf)
        content_dict[file_name] = whole_pdf

    return content_dict

In [4]:
files_list

['34944_高鐵_促參.pdf',
 '38005_核四_政策.pdf',
 '39243_核四_品質.pdf',
 '39477_核四_料件.pdf',
 '46365_高鐵_基金.pdf',
 '48052_大客車_逃生門.pdf',
 '48746_大客車_安全門.pdf',
 '49490_核四_延宕.pdf',
 '49627_核四_停建.pdf',
 '49676_大客車_駕照.pdf',
 '52922_大客車_超時.pdf',
 '54210_高鐵_出資.pdf',
 '54376_高鐵_航發.pdf',
 '54561_高鐵_機電.pdf',
 '58930_核四_封存.pdf']

In [5]:
pdf_contents = read_pdf(files_list)

In [6]:
def content_extraction(input, pattern):

    # Search using the pattern
    match = re.search(pattern, input, re.S)

    if match:
        result = match.group(1).strip()  # Use strip() to remove any leading/trailing whitespace
        print("Extraction succeed.")
        return result
    else:
        print("No match found")
        pass

In [7]:
pattern = r"貳、案   由(.*?)參、事實與理由"

extracted_content = {}
for filename, content in pdf_contents.items():
    extracted_content[filename] = content_extraction(content, pattern)

Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.


In [40]:
extracted_content

{'34944_高鐵_促參.pdf': '交通部於 87年間與臺灣高速鐵路公司簽訂「臺灣南北高速鐵路興建營運合約」及「臺灣南北高速鐵路站區開發合約 」，疏未預先審度該公司展延高鐵通車營運時程及遲延受領站區用地所生 損失之責任歸屬，並綢繆約定 相關處罰條款 ，嗣對政府權益保障恝置不察，一再同意該公司展延通車營運時程及受領站區用地， 致政府蒙受回饋金與租金收入減少及顧問費用增加等鉅額損失，均有違失，爰依監察法第 24條規定提案糾正',
 '38005_核四_政策.pdf': '核四封存後每年仍耗費數億元於資產維護管理，行政院及經濟部對外宣告核四重啟不可行，核四興建費用 2,833億元頇列為損失，行政院及 經濟部對核四政策之重大變動，導致資源嚴重浪費； 再者，經濟部宣布能源配比 (燃氣50%、燃煤30%、再生能源20%)之能源轉型政策，未經能源安全、能源經濟及環境影響等完整評估 ，復於再生能源發電量增加有限情況下， 以運轉中核電機組長期停機 方式減核 ，致近年火力發購電量逐年提高， 106年占比達84.4%，燃煤發電增幅 甚至高於燃氣，造成嚴重空氣污染；以及經濟部宣布 新能源政策 之前，並未評估其對電價之影響，迄 106年3月行政院始於國公營企業體檢小組會議評估 等情，均有違失，爰依法提案糾正',
 '39243_核四_品質.pdf': '台電公司 未落實「核四工程品質保證方案」，致龍門電廠試運轉時違規與注意改善事項層出不窮，如 抑壓池灌水作業 不當，致反應器廠房底層淹水 、壓力試驗合格之室內消防栓 系統，其 太平龍頭 竟脫落，致汽機廠房積水等 ，均嚴重衝擊國人對核能安全運轉之信心等情 ，確有諸多違失，爰依法提案糾正',
 '39477_核四_料件.pdf': '台灣電力股份有限公司 (下稱台電公司 )於本院調查核四廠一號機因施工測詴期間設備損壞而移用二號機相關設備之過程，所提供資料內容前後不一，設備組件損壞、採購及修復個數未能確實清查正確，顯見台電公司核四廠之料件管理系統紊亂，且回復本院公文一再發生資料正確性不足，核有怠失；另台電公司於 81年陳報核四興建計畫， 竟以69年所估算成本陳報， 未能如實報告核四建廠成本，致使政府無法確實評估該項投資計畫之成本效益，台電公司表示當時即考慮日後再以追加預算方式提出，此亦導致政 府長年來不得不對核四預算持續加碼，形成台電公司及國家

In [9]:
import re

def extract_label(filename):
    # Assuming the label is always after the first underscore and ends before the second underscore
    match = re.search(r'_(.*?)_', filename)
    if match:
        return match.group(1)  # This extracts text between the first pair of underscores
    return None  # In case the pattern does not match

In [41]:
formatted_data  = [{'label': extract_label(filename), 'text' : content } for filename, content in extracted_content.items()]
formatted_data 

[{'label': '高鐵',
  'text': '交通部於 87年間與臺灣高速鐵路公司簽訂「臺灣南北高速鐵路興建營運合約」及「臺灣南北高速鐵路站區開發合約 」，疏未預先審度該公司展延高鐵通車營運時程及遲延受領站區用地所生 損失之責任歸屬，並綢繆約定 相關處罰條款 ，嗣對政府權益保障恝置不察，一再同意該公司展延通車營運時程及受領站區用地， 致政府蒙受回饋金與租金收入減少及顧問費用增加等鉅額損失，均有違失，爰依監察法第 24條規定提案糾正'},
 {'label': '核四',
  'text': '核四封存後每年仍耗費數億元於資產維護管理，行政院及經濟部對外宣告核四重啟不可行，核四興建費用 2,833億元頇列為損失，行政院及 經濟部對核四政策之重大變動，導致資源嚴重浪費； 再者，經濟部宣布能源配比 (燃氣50%、燃煤30%、再生能源20%)之能源轉型政策，未經能源安全、能源經濟及環境影響等完整評估 ，復於再生能源發電量增加有限情況下， 以運轉中核電機組長期停機 方式減核 ，致近年火力發購電量逐年提高， 106年占比達84.4%，燃煤發電增幅 甚至高於燃氣，造成嚴重空氣污染；以及經濟部宣布 新能源政策 之前，並未評估其對電價之影響，迄 106年3月行政院始於國公營企業體檢小組會議評估 等情，均有違失，爰依法提案糾正'},
 {'label': '核四',
  'text': '台電公司 未落實「核四工程品質保證方案」，致龍門電廠試運轉時違規與注意改善事項層出不窮，如 抑壓池灌水作業 不當，致反應器廠房底層淹水 、壓力試驗合格之室內消防栓 系統，其 太平龍頭 竟脫落，致汽機廠房積水等 ，均嚴重衝擊國人對核能安全運轉之信心等情 ，確有諸多違失，爰依法提案糾正'},
 {'label': '核四',
  'text': '台灣電力股份有限公司 (下稱台電公司 )於本院調查核四廠一號機因施工測詴期間設備損壞而移用二號機相關設備之過程，所提供資料內容前後不一，設備組件損壞、採購及修復個數未能確實清查正確，顯見台電公司核四廠之料件管理系統紊亂，且回復本院公文一再發生資料正確性不足，核有怠失；另台電公司於 81年陳報核四興建計畫， 竟以69年所估算成本陳報， 未能如實報告核四建廠成本，致使政府無法確實評估該項投資計畫之成本效益，台電公司表示當時即考慮日後再以追加預算方式提出，此

In [42]:
label = []
for i in range(len(formatted_data)):
    current_label = formatted_data[i]['label']
    if current_label not in label:
        label.append(current_label)
    else:
        continue
label_feature = ClassLabel(names=label)
label_feature

ClassLabel(names=['高鐵', '核四', '大客車'], id=None)

In [43]:
dataset = Dataset.from_dict({
    'text': [item['text'] for item in formatted_data],
    'label': [item['label'] for item in formatted_data]
})


In [44]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 15
})

In [45]:
def preprocess_label(example):
    
    example['label'] = [label_feature.str2int(label) for label in example['label']]
    return example

In [46]:
dataset_processed = dataset.map(preprocess_label, batched=True)
dataset_split = dataset_processed.train_test_split(test_size=0.3)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [47]:
dataset_split

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5
    })
})

## sentence-transformers/all-MiniLM-L6-v2

In [17]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
model_ckpt = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [19]:
sentences = ['前天是禮拜二', '昨天是禮拜二']
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
encoded_input

{'input_ids': tensor([[ 101, 1776, 1811,  100,  100,  100, 1752,  102],
        [ 101,  100, 1811,  100,  100,  100, 1752,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [20]:
with torch.inference_mode():
    model_output = model(**encoded_input)

In [21]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [22]:
# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

In [23]:
cosine_similarity(sentence_embeddings[0].reshape(1, -1), sentence_embeddings[1].reshape(1, -1))

array([[0.92570424]], dtype=float32)

In [24]:
tokenized_dict = {}
for filename, content in extracted_content.items():
    encoded_content = tokenizer(content, padding=True, truncation=True, return_tensors='pt')
    with torch.inference_mode():
        model_output = model(**encoded_content)
    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_content['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    tokenized_dict[filename] = sentence_embeddings

In [25]:
similarity_df = pd.DataFrame(index=tokenized_dict.keys(), columns=tokenized_dict.keys())

In [26]:
# Create an empty DataFrame
similarity_df = pd.DataFrame(index=tokenized_dict.keys(), columns=tokenized_dict.keys())

# Calculate cosine similarity between each pair of vectors
for name1, vec1 in tokenized_dict.items():
    for name2, vec2 in tokenized_dict.items():
        similarity_df.loc[name1, name2] = cosine_similarity(vec1, vec2)[0][0]

In [27]:
similarity_df

,34944_高鐵_促參.pdf,38005_核四_政策.pdf,39243_核四_品質.pdf,39477_核四_料件.pdf,46365_高鐵_基金.pdf,48052_大客車_逃生門.pdf,48746_大客車_安全門.pdf,49490_核四_延宕.pdf,49627_核四_停建.pdf,49676_大客車_駕照.pdf,52922_大客車_超時.pdf,54210_高鐵_出資.pdf,54376_高鐵_航發.pdf,54561_高鐵_機電.pdf,58930_核四_封存.pdf
34944_高鐵_促參.pdf,1.0,0.771646,0.715157,0.810293,0.757769,0.724102,0.737007,0.831104,0.843057,0.820516,0.881772,0.808,0.81963,0.696251,0.693054
38005_核四_政策.pdf,0.771646,1.0,0.5546,0.769547,0.676038,0.573962,0.77681,0.841877,0.869432,0.682205,0.759324,0.786793,0.833689,0.545257,0.829629
39243_核四_品質.pdf,0.715157,0.5546,1.0,0.638359,0.575538,0.738709,0.514832,0.668872,0.666733,0.613016,0.701931,0.53878,0.608017,0.680304,0.52084
39477_核四_料件.pdf,0.810293,0.769547,0.638359,1.0,0.569365,0.558339,0.700211,0.87964,0.864525,0.762747,0.870343,0.848424,0.852757,0.476027,0.704625
46365_高鐵_基金.pdf,0.757769,0.676038,0.575538,0.569365,1.0,0.562911,0.567731,0.649495,0.647114,0.580461,0.615131,0.614591,0.653924,0.599685,0.605182
48052_大客車_逃生門.pdf,0.724102,0.573962,0.738709,0.558339,0.562911,1.0,0.66724,0.631306,0.607808,0.745443,0.694181,0.489822,0.543012,0.774908,0.499585
48746_大客車_安全門.pdf,0.737007,0.77681,0.514832,0.700211,0.567731,0.66724,1.0,0.783596,0.730051,0.839712,0.767071,0.732613,0.744173,0.559655,0.742649
49490_核四_延宕.pdf,0.831104,0.841877,0.668872,0.87964,0.649495,0.631306,0.783596,1.0,0.913284,0.808667,0.870392,0.842519,0.865204,0.556715,0.786616
49627_核四_停建.pdf,0.843057,0.869432,0.666733,0.864525,0.647114,0.607808,0.730051,0.913284,1.0,0.740321,0.842977,0.853902,0.885557,0.599092,0.808137
49676_大客車_駕照.pdf,0.820516,0.682205,0.613016,0.762747,0.580461,0.745443,0.839712,0.808667,0.740321,1.0,0.841511,0.707834,0.727746,0.638968,0.687937


## distilbert-base-uncased

In [48]:
from transformers import AutoModel
from transformers import Trainer, TrainingArguments


In [49]:
model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

In [30]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [31]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [50]:
dataset_encoded = dataset_split.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [71]:
dataset_encoded.set_format("torch", 
                            columns=["input_ids",  "label","attention_mask"])

In [72]:
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5
    })
})

In [54]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    label = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label, preds, average="weighted")
    acc = accuracy_score(label, preds)
    return {"accuracy": acc, "f1": f1}

In [56]:
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5
    })
})

In [75]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer

In [101]:
model = DistilBertForSequenceClassification.from_pretrained(model_ckpt, num_labels=3)

batch_size = 1
logging_steps = len(dataset_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=20,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False, 
                                  log_level="error")

trainer = Trainer(model=model, args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=dataset_encoded["train"],
                  eval_dataset=dataset_encoded["test"],
                  tokenizer=tokenizer)

# Now you can train the model
trainer.train()


  0%|          | 0/200 [00:00<?, ?it/s]

{'loss': 1.1551, 'grad_norm': 8.319483757019043, 'learning_rate': 1.9e-05, 'epoch': 1.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.1386288404464722, 'eval_accuracy': 0.2, 'eval_f1': 0.06666666666666667, 'eval_runtime': 1.039, 'eval_samples_per_second': 4.812, 'eval_steps_per_second': 4.812, 'epoch': 1.0}
{'loss': 1.1296, 'grad_norm': 6.946297645568848, 'learning_rate': 1.8e-05, 'epoch': 2.0}


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.0705875158309937, 'eval_accuracy': 0.4, 'eval_f1': 0.3, 'eval_runtime': 1.2075, 'eval_samples_per_second': 4.141, 'eval_steps_per_second': 4.141, 'epoch': 19.0}
{'loss': 0.3382, 'grad_norm': 7.091785907745361, 'learning_rate': 0.0, 'epoch': 20.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.062636375427246, 'eval_accuracy': 0.4, 'eval_f1': 0.3, 'eval_runtime': 1.1825, 'eval_samples_per_second': 4.228, 'eval_steps_per_second': 4.228, 'epoch': 20.0}
{'train_runtime': 263.1584, 'train_samples_per_second': 0.76, 'train_steps_per_second': 0.76, 'train_loss': 0.740736973285675, 'epoch': 20.0}


TrainOutput(global_step=200, training_loss=0.740736973285675, metrics={'train_runtime': 263.1584, 'train_samples_per_second': 0.76, 'train_steps_per_second': 0.76, 'total_flos': 23078716178400.0, 'train_loss': 0.740736973285675, 'epoch': 20.0})

In [102]:
trainer.save_model("./models/DistilBertForSequenceClassification_finetune")
tokenizer.save_pretrained("./models/DistilBertForSequenceClassification_finetune")

('./models/DistilBertForSequenceClassification_finetune\\tokenizer_config.json',
 './models/DistilBertForSequenceClassification_finetune\\special_tokens_map.json',
 './models/DistilBertForSequenceClassification_finetune\\vocab.txt',
 './models/DistilBertForSequenceClassification_finetune\\added_tokens.json')

In [103]:
# Load the model and tokenizer for classification
model_name = "./models/DistilBertForSequenceClassification_finetune"
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model.eval()


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [118]:
def get_embeddings(texts, model, tokenizer):
    # Tokenize the input texts
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    # Pass inputs through the model to get hidden states
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    # Get the embeddings from the last hidden state
    hidden_states = outputs.hidden_states[-1]
    # Average the hidden states to get a single vector representation for each text
    embeddings = hidden_states.mean(dim=1)
    return embeddings


In [119]:
# Extract texts from the dictionary
texts = list(extracted_content.values())

# Get embeddings for all documents
embeddings = get_embeddings(texts, model, tokenizer)

# Compute cosine similarity between all pairs of embeddings
cosine_similarities = cosine_similarity(embeddings)

# Convert cosine similarity matrix to a DataFrame
doc_keys = list(extracted_content.keys())
cosine_sim_df = pd.DataFrame(cosine_similarities, index=doc_keys, columns=doc_keys)


In [120]:
cosine_sim_df

,34944_高鐵_促參.pdf,38005_核四_政策.pdf,39243_核四_品質.pdf,39477_核四_料件.pdf,46365_高鐵_基金.pdf,48052_大客車_逃生門.pdf,48746_大客車_安全門.pdf,49490_核四_延宕.pdf,49627_核四_停建.pdf,49676_大客車_駕照.pdf,52922_大客車_超時.pdf,54210_高鐵_出資.pdf,54376_高鐵_航發.pdf,54561_高鐵_機電.pdf,58930_核四_封存.pdf
34944_高鐵_促參.pdf,1.000000,0.962280,0.974743,0.980940,0.983955,0.961192,0.957041,0.949913,0.966856,0.986660,0.981854,0.918519,0.967689,0.894094,0.968567
38005_核四_政策.pdf,0.962280,1.000000,0.937627,0.981116,0.942181,0.899119,0.978538,0.981397,0.989672,0.976286,0.962156,0.954688,0.978305,0.824177,0.983393
39243_核四_品質.pdf,0.974743,0.937627,1.000000,0.952167,0.974248,0.967946,0.924130,0.923296,0.939098,0.963191,0.958713,0.866308,0.932105,0.894870,0.951001
39477_核四_料件.pdf,0.980940,0.981116,0.952167,1.000000,0.962773,0.928281,0.985181,0.981115,0.990165,0.989079,0.985264,0.960494,0.988859,0.833852,0.988852
46365_高鐵_基金.pdf,0.983955,0.942181,0.974248,0.962773,1.000000,0.966325,0.936774,0.930900,0.942926,0.968626,0.965432,0.896264,0.949064,0.912097,0.948836
48052_大客車_逃生門.pdf,0.961192,0.899119,0.967946,0.928281,0.966325,1.000000,0.883831,0.874687,0.900912,0.944919,0.942805,0.820494,0.897106,0.907426,0.910080
48746_大客車_安全門.pdf,0.957041,0.978538,0.924130,0.985181,0.936774,0.883831,1.000000,0.989947,0.990211,0.979068,0.970056,0.979183,0.991997,0.790399,0.983443
49490_核四_延宕.pdf,0.949913,0.981397,0.923296,0.981115,0.930900,0.874687,0.989947,1.000000,0.991793,0.973724,0.967373,0.972944,0.988754,0.784708,0.986784
49627_核四_停建.pdf,0.966856,0.989672,0.939098,0.990165,0.942926,0.900912,0.990211,0.991793,1.000000,0.983878,0.977314,0.966638,0.989626,0.803488,0.993448
49676_大客車_駕照.pdf,0.986660,0.976286,0.963191,0.989079,0.968626,0.944919,0.979068,0.973724,0.983878,1.000000,0.991682,0.938168,0.981119,0.847832,0.985918


In [128]:
cosine_sim_df['49490_核四_延宕.pdf'].sort_values(ascending=False)

49490_核四_延宕.pdf      1.000000
49627_核四_停建.pdf      0.991793
48746_大客車_安全門.pdf    0.989947
54376_高鐵_航發.pdf      0.988754
58930_核四_封存.pdf      0.986784
38005_核四_政策.pdf      0.981397
39477_核四_料件.pdf      0.981115
49676_大客車_駕照.pdf     0.973724
54210_高鐵_出資.pdf      0.972944
52922_大客車_超時.pdf     0.967373
34944_高鐵_促參.pdf      0.949913
46365_高鐵_基金.pdf      0.930900
39243_核四_品質.pdf      0.923296
48052_大客車_逃生門.pdf    0.874687
54561_高鐵_機電.pdf      0.784708
Name: 49490_核四_延宕.pdf, dtype: float32

In [124]:
cosine_sim_df.to_csv('cos_sim_v1.csv', encoding='big5')

## paraphrase-MiniLM-L6-v2

In [141]:
from transformers import AutoModel, AutoTokenizer
import torch

In [146]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def get_embeddings(texts, model, tokenizer):
    # Tokenize the input texts
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

    # Pass inputs through the model to get hidden states
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    return outputs

In [147]:
# Extract texts from the dictionary
texts = list(extracted_content.values())

# Get embeddings for all documents
embeddings = get_embeddings(texts, model, tokenizer)

# normalized embedding
norm_embeddings  = mean_pooling(embeddings, encoded_input['attention_mask'])

# Compute cosine similarity between all pairs of embeddings
cosine_similarities = cosine_similarity(norm_embeddings)

# Convert cosine similarity matrix to a DataFrame
doc_keys = list(extracted_content.keys())
cosine_sim_df = pd.DataFrame(cosine_similarities, index=doc_keys, columns=doc_keys)


start
end


RuntimeError: The expanded size of the tensor (446) must match the existing size (8) at non-singleton dimension 1.  Target sizes: [15, 446, 384].  Tensor sizes: [2, 8, 1]

In [152]:
texts[4]

'近年利率逐年下調， 資金成本下降， 惟交通部未督飭高公局利用此 改變趨勢而進行財務調度，積極規劃國道基金債務之舉新還舊，減輕利息 負擔，改善財務結構 ；該部復坐視國道基金之財務操作 受高鐵基金之影響；又該部迄今未將 國道基金負擔 未具自償性且與國道並無直接關聯之地區性道路興建支出，撥還該基金等情，均核有重大違失，爰依法提案糾正'

In [153]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = list(extracted_content.values())

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[-0.2189,  0.8674,  0.4606,  ..., -0.6580, -0.4033,  0.6827],
        [ 0.0677,  0.7340,  0.7409,  ..., -0.4495, -0.2585,  0.5965],
        [-0.5176,  0.7155,  0.3799,  ..., -0.7914, -0.3436,  0.7034],
        ...,
        [-0.0959,  0.8173,  0.6737,  ..., -0.6921, -0.6189,  0.6678],
        [-0.2214,  0.6630,  0.3330,  ..., -0.8624,  0.2184,  0.6604],
        [ 0.0169,  0.9363,  0.4728,  ..., -0.5107, -0.2639,  0.6777]])


In [156]:
cosine_similarities = cosine_similarity(sentence_embeddings)

In [157]:
doc_keys = list(extracted_content.keys())
cosine_sim_df = pd.DataFrame(cosine_similarities, index=doc_keys, columns=doc_keys)

In [163]:
cosine_sim_df['48052_大客車_逃生門.pdf'].sort_values(ascending=False)

48052_大客車_逃生門.pdf    1.000000
39243_核四_品質.pdf      0.949153
54561_高鐵_機電.pdf      0.939176
46365_高鐵_基金.pdf      0.906380
34944_高鐵_促參.pdf      0.888729
52922_大客車_超時.pdf     0.878784
39477_核四_料件.pdf      0.874396
49676_大客車_駕照.pdf     0.863293
54376_高鐵_航發.pdf      0.849896
49627_核四_停建.pdf      0.848142
54210_高鐵_出資.pdf      0.836646
49490_核四_延宕.pdf      0.791983
58930_核四_封存.pdf      0.791870
38005_核四_政策.pdf      0.789064
48746_大客車_安全門.pdf    0.784512
Name: 48052_大客車_逃生門.pdf, dtype: float32

In [162]:
extracted_content

{'34944_高鐵_促參.pdf': '交通部於 87年間與臺灣高速鐵路公司簽訂「臺灣南北高速鐵路興建營運合約」及「臺灣南北高速鐵路站區開發合約 」，疏未預先審度該公司展延高鐵通車營運時程及遲延受領站區用地所生 損失之責任歸屬，並綢繆約定 相關處罰條款 ，嗣對政府權益保障恝置不察，一再同意該公司展延通車營運時程及受領站區用地， 致政府蒙受回饋金與租金收入減少及顧問費用增加等鉅額損失，均有違失，爰依監察法第 24條規定提案糾正',
 '38005_核四_政策.pdf': '核四封存後每年仍耗費數億元於資產維護管理，行政院及經濟部對外宣告核四重啟不可行，核四興建費用 2,833億元頇列為損失，行政院及 經濟部對核四政策之重大變動，導致資源嚴重浪費； 再者，經濟部宣布能源配比 (燃氣50%、燃煤30%、再生能源20%)之能源轉型政策，未經能源安全、能源經濟及環境影響等完整評估 ，復於再生能源發電量增加有限情況下， 以運轉中核電機組長期停機 方式減核 ，致近年火力發購電量逐年提高， 106年占比達84.4%，燃煤發電增幅 甚至高於燃氣，造成嚴重空氣污染；以及經濟部宣布 新能源政策 之前，並未評估其對電價之影響，迄 106年3月行政院始於國公營企業體檢小組會議評估 等情，均有違失，爰依法提案糾正',
 '39243_核四_品質.pdf': '台電公司 未落實「核四工程品質保證方案」，致龍門電廠試運轉時違規與注意改善事項層出不窮，如 抑壓池灌水作業 不當，致反應器廠房底層淹水 、壓力試驗合格之室內消防栓 系統，其 太平龍頭 竟脫落，致汽機廠房積水等 ，均嚴重衝擊國人對核能安全運轉之信心等情 ，確有諸多違失，爰依法提案糾正',
 '39477_核四_料件.pdf': '台灣電力股份有限公司 (下稱台電公司 )於本院調查核四廠一號機因施工測詴期間設備損壞而移用二號機相關設備之過程，所提供資料內容前後不一，設備組件損壞、採購及修復個數未能確實清查正確，顯見台電公司核四廠之料件管理系統紊亂，且回復本院公文一再發生資料正確性不足，核有怠失；另台電公司於 81年陳報核四興建計畫， 竟以69年所估算成本陳報， 未能如實報告核四建廠成本，致使政府無法確實評估該項投資計畫之成本效益，台電公司表示當時即考慮日後再以追加預算方式提出，此亦導致政 府長年來不得不對核四預算持續加碼，形成台電公司及國家